In [2]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
%matplotlib inline

### Static Variables

In [3]:
linefiles = os.listdir("bus_data/clean_data2")
lines = []
for line in linefiles:
    lines.append(line.replace(".csv",""))
    


### Extracting Data

### Read in External Data

In [4]:
schedule = pd.read_csv("bus_data/dublinbus_scheduledData2013csv.csv", low_memory=False, encoding="ISO-8859-1")

In [5]:
schedule.head()

,trip_id,arrival_time,departure_time,stop_id,Name without locality,Name,Lat,Long,stop_sequence,stop_headsign,shape_dist_traveled
0,1.1598.0-33A-b12-1.451.I,16:15:00,16:15:00,8240DB003813,Footbridge,Skerries Road,53.591377,-6.156176,1,Swords,0.000000
1,1.1598.0-33A-b12-1.451.I,16:15:39,16:15:39,8240DB003814,Barnageeragh Road,Balbriggan Rd,53.585544,-6.148605,2,Swords,827.070328
2,1.1598.0-33A-b12-1.451.I,16:17:11,16:17:11,8240DB003815,Barnageeragh Road,North Cliff Heights,53.581935,-6.125247,3,Swords,2746.649492
3,1.1598.0-33A-b12-1.451.I,16:17:22,16:17:22,8240DB003816,Mourne Drive,North Cliff Heights,53.580471,-6.127622,4,Swords,2973.900755
4,1.1598.0-33A-b12-1.451.I,16:17:44,16:17:44,8240DB003566,Selskar Court,Town Parks Rd,53.578067,-6.125055,5,Swords,3447.155571


In [6]:
#get source
#only trips with stop sequence 42


In [7]:
# Dropping null values - some in our dataframe are marked as 'null' (string) for some reason?
schedule = schedule.dropna()
schedule = schedule[schedule.stop_id != 'null']

### Exploring schedule

In [8]:
# Extracting StopID from stop_id

def get_stop(string):
    var = string[-4:]
    return var

schedule["StopID"] = schedule['stop_id'].apply(lambda x: get_stop(x))

In [9]:
# Extracting LineID from trip_id
schedule["LineID"] = schedule["trip_id"].str.extract('\-(.*?)\-')

C:\Users\eogha\Anaconda3\envs\comp47350\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app


In [10]:
# Convert StopIDs back to int
# df['StopID'] = df['StopID'].astype('int64')
schedule['StopID'] = schedule['StopID'].astype('int64')

In [11]:
adf = schedule[['StopID','LineID','stop_headsign','stop_sequence']]
adf[(adf.LineID == "1") & (adf.stop_headsign == "Sandymount")].drop_duplicates().stop_sequence.value_counts()

21    2
20    2
18    2
17    2
16    2
15    2
14    2
13    2
12    2
11    2
10    2
9     2
8     2
7     2
6     2
5     2
4     2
3     2
2     2
19    2
1     2
41    1
32    1
40    1
39    1
38    1
37    1
36    1
35    1
34    1
33    1
31    1
22    1
30    1
29    1
28    1
27    1
26    1
25    1
24    1
23    1
42    1
Name: stop_sequence, dtype: int64

In [12]:
# Extracting direction from trip_id
def get_direction(string):
    var = string[-1:]
    return var

schedule["Direction"] = schedule['trip_id'].apply(lambda x: get_direction(x))

In [13]:
schedule = schedule[(schedule.Direction == "I") | (schedule.Direction == "O")]    

In [14]:
# Converting Direction to JourneyPatternID
schedule["JourneyPatternID"] = schedule['Direction'].apply(lambda x: "1001" if x == "I" else "0001")
schedule['JourneyPatternID'] = schedule['JourneyPatternID'].astype('int64')

In [65]:
schedule15 = schedule[(schedule.LineID == "1") & (schedule.JourneyPatternID == 1)]

In [66]:
schedule15.stop_headsign.unique()

array(['Sandymount', "O'Connell Street"], dtype=object)

In [67]:
schedule15[schedule15.stop_headsign == "Sandymount"].drop_duplicates().stop_sequence.value_counts()

4     114
21    114
13    114
9     114
5     114
1     114
2     114
6     114
10    114
14    114
18    114
17    114
20    114
3     114
7     114
11    114
15    114
19    114
16    114
12    114
8     114
28    103
24    103
36    103
40    103
32    103
39    103
25    103
29    103
33    103
37    103
35    103
22    103
26    103
30    103
34    103
38    103
42    103
23    103
27    103
31    103
41    103
Name: stop_sequence, dtype: int64

In [74]:
firstlines = schedule15.groupby(['LineID','stop_headsign','JourneyPatternID']).head(1)

In [75]:
origins = firstlines.StopID.unique()

In [76]:
origins

array([226], dtype=int64)

In [ ]:
adf['source'] = adf

In [62]:
adf = schedule #[['StopID','LineID','stop_headsign','stop_sequence', 'JourneyPatternID']]
adf[(adf.LineID == "1") & (adf.stop_headsign == "Sandymount") & (adf.JourneyPatternID == 1)].drop_duplicates().stop_sequence.value_counts()

Series([], Name: stop_sequence, dtype: int64)

In [156]:
# dropping irrelevant columns
for column in ['trip_id', 'stop_id', 'arrival_time', 'departure_time', 'shape_dist_traveled', 'Direction']:
        schedule = schedule.drop(column, 1)

In [157]:
# merge_schedule = schedule[['Name without locality','Name','Lat','Long','StopID']]

In [158]:
# merge_schedule.shape

In [159]:
# schedule.head()

## Single Extraction
    

In [160]:
headsigns = pd.read_csv("Headsigns.csv", low_memory=False, encoding="ISO-8859-1")
headsigns = headsigns.drop('Unnamed: 0', 1)

In [161]:
headsigns.head()

,JourneyPatternID,Headsign,LineID
0,1,Sandymount,1
1,1001,O'Connell Street,1
2,1001,Station Rd,102
3,1,Airport,102
4,1001,Clontarf Road,104


In [162]:
routes = headsigns[['JourneyPatternID','LineID']]

In [163]:
routes.head()

,JourneyPatternID,LineID
0,1,1
1,1001,1
2,1001,102
3,1,102
4,1001,104


In [164]:
# stops = pd.read_csv("stops.csv", low_memory=False, encoding="ISO-8859-1")
# stops = stops.drop('Unnamed: 0', 1)

In [165]:
# stops.head()

In [166]:
schedule.head()

,Name without locality,Name,Lat,Long,stop_sequence,stop_headsign,StopID,LineID,JourneyPatternID
0,Footbridge,Skerries Road,53.591377,-6.156176,1,Swords,3813,33A,1001
1,Barnageeragh Road,Balbriggan Rd,53.585544,-6.148605,2,Swords,3814,33A,1001
2,Barnageeragh Road,North Cliff Heights,53.581935,-6.125247,3,Swords,3815,33A,1001
3,Mourne Drive,North Cliff Heights,53.580471,-6.127622,4,Swords,3816,33A,1001
4,Selskar Court,Town Parks Rd,53.578067,-6.125055,5,Swords,3566,33A,1001


In [167]:
df = pd.merge(routes, schedule, how='inner', on=['LineID','JourneyPatternID'])

In [168]:
adf = df[['StopID','LineID','stop_headsign','stop_sequence']]
adf[(adf.LineID == "1") & (adf.stop_headsign == "Sandymount")].stop_sequence.drop_duplicates().value_counts()

42    1
11    1
19    1
18    1
17    1
16    1
15    1
14    1
13    1
12    1
10    1
41    1
9     1
8     1
7     1
6     1
5     1
4     1
3     1
2     1
20    1
21    1
22    1
23    1
40    1
39    1
38    1
37    1
36    1
35    1
34    1
33    1
32    1
31    1
30    1
29    1
28    1
27    1
26    1
25    1
24    1
1     1
Name: stop_sequence, dtype: int64

In [173]:
# tdf = schedule[(schedule.LineID == "1") & (schedule.JourneyPatternID == 1)].drop_duplicates()
tdf = schedule[(schedule.LineID == "1") & (schedule.stop_headsign == "Sandymount")].drop_duplicates()

In [178]:
adf = tdf[['StopID','LineID','stop_headsign', 'JourneyPatternID', 'stop_sequence']]
# adf[(adf.LineID == "1") & (adf.stop_headsign == "Sandymount")].stop_sequence.drop_duplicates().value_counts()
adf[(adf.LineID == "1") & (adf.JourneyPatternID == 1)].drop_duplicates().stop_sequence.value_counts()

21    2
20    2
18    2
17    2
16    2
15    2
14    2
13    2
12    2
11    2
10    2
9     2
8     2
7     2
6     2
5     2
4     2
3     2
2     2
19    2
1     2
41    1
32    1
40    1
39    1
38    1
37    1
36    1
35    1
34    1
33    1
31    1
22    1
30    1
29    1
28    1
27    1
26    1
25    1
24    1
23    1
42    1
Name: stop_sequence, dtype: int64

In [170]:
tdf.head()

,Name without locality,Name,Lat,Long,stop_sequence,stop_headsign,StopID,LineID,JourneyPatternID
82414,Shanard Avenue,Shanard Road,53.391140,-6.262185,1,Sandymount,226,1,1
82415,Oldtown Avenue,Shanliss Rd,53.391852,-6.259796,2,Sandymount,228,1,1
82416,Oldtown Road,Shanliss Rd,53.391399,-6.256521,3,Sandymount,229,1,1
82417,Shanliss Drive,Shanliss Rd,53.391143,-6.251314,4,Sandymount,227,1,1
82418,Shanowen Road,Shanliss Rd,53.389888,-6.249050,5,Sandymount,230,1,1


In [171]:
tdf.stop_headsign.unique()

array(['Sandymount'], dtype=object)

In [172]:
# tdf[(tdf.stop_sequence == 23)].drop_duplicates()
tdf.drop_duplicates().stop_sequence.value_counts()


21    2
20    2
18    2
17    2
16    2
15    2
14    2
13    2
12    2
11    2
10    2
9     2
8     2
7     2
6     2
5     2
4     2
3     2
2     2
19    2
1     2
41    1
32    1
40    1
39    1
38    1
37    1
36    1
35    1
34    1
33    1
31    1
22    1
30    1
29    1
28    1
27    1
26    1
25    1
24    1
23    1
42    1
Name: stop_sequence, dtype: int64

In [ ]:
tempdf = df[(df.LineID == "1") & (df.JourneyPatternID == 1)].drop_duplicates()

In [90]:
tempdf.drop_duplicates().stop_sequence.value_counts()
# tempdf[(tempdf.stop_sequence == 21)].drop_duplicates()

21    3
20    3
18    3
17    3
16    3
15    3
14    3
13    3
12    3
11    3
10    3
9     3
8     3
7     3
6     3
5     3
4     3
3     3
2     3
19    3
1     3
23    2
22    2
41    1
33    1
40    1
39    1
38    1
37    1
36    1
35    1
34    1
31    1
32    1
30    1
29    1
28    1
27    1
26    1
25    1
24    1
42    1
Name: stop_sequence, dtype: int64

In [64]:
tempdf[(tempdf.stop_sequence == "1")]

C:\Users\eogha\Anaconda3\envs\comp47350\lib\site-packages\pandas\core\ops.py:792: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


TypeError: invalid type comparison

# Single Extraction

In [168]:
line = lines[1]

#Iterate through line data files
# for line in lines:
df = pd.read_csv("bus_data/clean_data2/" + line + ".csv", low_memory=False, header=None)
df.columns = ["Timestamp", "LineID", "JourneyPatternID", "TimeFrame", 
              "VehicleJourneyID", "Lon", "Lat", "VehicleID", "StopID", 
              "AtStop", "HumanTime", "Day", "Hour", "JourneyGroup", "Runtime"]

line

'102'

In [169]:
#all stops for this line
stoplist = df['StopID']
stoplist.head()

0    7348
1    3669
2    3672
3    3078
4    3087
Name: StopID, dtype: object

In [170]:
stoplist.shape

(146384,)

In [171]:
# df['StopID'] = df['StopID'].astype('str')
# df.StopID = df.StopID.astype("int64")
# df = pd.DataFrame(df, cols = ['StopID'])

#Drop Duplicates
stoplist = stoplist.drop_duplicates()

In [172]:
list(stoplist)
for i in stoplist:
    stops.append(i)

In [173]:
# stops

In [174]:
#make stops dataframe from all non duplicated stops occurring in our data
all_stops = pd.DataFrame(stoplist, columns=['StopID'])
clean_stops = all_stops.drop_duplicates()

In [176]:
clean_stops.head()

,StopID,Name without locality,Name,Lat,Long


In [177]:
clean_stops.shape

(0, 5)

In [178]:
master_stops = master_stops.append(clean_stops)

In [179]:
# master_stops.shape

## Get Stops Looped

In [186]:
all_stops = []

#     line = lines[0]
for line in lines:
    #Iterate through line data files
    # for line in lines:
    df = pd.read_csv("bus_data/clean_data2/" + line + ".csv", low_memory=False, header=None)
    df.columns = ["Timestamp", "LineID", "JourneyPatternID", "TimeFrame", 
                  "VehicleJourneyID", "Lon", "Lat", "VehicleID", "StopID", 
                  "AtStop", "HumanTime", "Day", "Hour", "JourneyGroup", "Runtime"]
#     stops = []

    line_stop_list = df['StopID']
    # df['StopID'] = df['StopID'].astype('str')
    # df.StopID = df.StopID.astype("int64")
    # df = pd.DataFrame(df, cols = ['StopID'])
    line_stop_list = line_stop_list.drop_duplicates()
    list(line_stop_list)
    for stop in line_stop_list:
        all_stops.append(stop)

stops = pd.DataFrame(all_stops, columns=['StopID'])
all_our_stops = stops.drop_duplicates()

clean_stops = pd.merge(all_our_stops, merge_schedule, how='inner', on=['StopID'])
clean_stops = clean_stops.drop_duplicates()

In [191]:
clean_stops.shape

(4219, 5)

In [193]:
final_stops = clean_stops.drop_duplicates('StopID')

In [203]:
final_stops = final_stops.reset_index()
final_stops = final_stops.drop('index', 1)

In [205]:
final_stops.shape

(4193, 5)

In [206]:
final_stops.to_csv("stops.csv")